# Smart Distribution Systems (B-KUL-H00P3A) 
Prof. Geert Deconinck

This exercise session will familiarize the students with basic concepts from machine learning and teach the students how these concepts can be used within power systems for forecasting.

The students will learn to:
* set up a machine learning environment, using state-of-the-art tools, such as keras, tensorflow and theano in Python;
* implement and train a two-layer neural network using Keras;
* use this neural network to make prediction about the wholesale electricity price.


In [ ]:
import pandas as pd
import numpy as np
%matplotlib ipympl
import matplotlib.pyplot as plt
import time
import pickle
import datetime
from datafetcher import EliaTotalLoadForecastFetcher, EliaWindFetcher, EliaSolarFetcher, ElexysBelpexFetcher
import datetime as dt
from sklearn.cross_validation import train_test_split, cross_val_predict, KFold

For more information on de pandas data frame see: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html

During this exercises we will focus on a certain part of the datasets. In 'data.ipynb' we saw that all datasets have relevant data from 2015 to 2017, so lets focus on this data range.

In [ ]:
start_date = dt.datetime(2015,1,1)
end_date = dt.datetime(2017,12,31)

In [ ]:
df_solar = EliaSolarFetcher().fetch().LoadFactor[start_date:end_date]
df_wind = EliaWindFetcher(only_cached=True).fetch().LoadFactor[start_date:end_date]

Unfortunately, in real-life datastes, sometimes, certain data is missing. We can see this by trying to assign the price to a variable named 'p' and catching the error if no value is available.

In [ ]:
df_belpex = ElexysBelpexFetcher().fetch().price
dates = pd.date_range(start=start_date, end=end_date, freq='1H')
for d in dates:
    try:
        p = df_belpex[d]
    except KeyError:
        print(d)

We will have to take assumptions of these values. In this case it makes sense to either assume they are the same as the previous hour or the next hour. Or, to take the average of both. In this case we will just assume the price is the same as the previous hour.

In [ ]:
dates = pd.date_range(start=start_date, end=end_date, freq='1H')
for d in dates:
    try:
        p = df_belpex[d]
    except KeyError:
        df_belpex.loc[d] = df_belpex[d-dt.timedelta(hours=1)]
df_belpex = df_belpex.sort_index()
df_belpex = df_belpex[start_date:end_date]

Additionally the wind dataset has certain NaN (Not a Number) values. We don't want these in our dataset either. Therefore, we choose to fill the previouos valid observation until the next valid one. <br>
Check for nan's in the wind dataset in the next cell!

In [ ]:
df_wind =  df_wind.fillna(method='pad')

Moreover, the different datasets have different time granularities. Different options are possible to deal with this issue. Here we will use the smallest granularity. Solar and PV data are on a 15 minute basis. Therefore, we will also put the belpex data on a 15 minutes basis (the price of every quarter within an hour is just the price of that hour). <br>
Check the pandas 'resample' documentation here: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html

In [ ]:
df_belpex = df_belpex.resample('15T').pad()
print(df_belpex[0:10])

Now we can all add them into one dataframe with 3 columns and a datetime index.

In [ ]:
d = {'belpex': df_belpex.values, 'solar': df_solar.values, 'wind': df_wind.values}
data = pd.DataFrame(index=df_belpex.index, data=d)
data.head()

Usefull commands
* data.head(10) prints the first 10 samples
* data.tail(10) prints the last 10 samples
* data.$wind$ prints the collumn with label $wind$ 
* data.wind.plot plots the wind timeseries

In [ ]:
data2 = pd.read_pickle('electricity_prices_belgium.pkl')

# 1. Visualize the data

In [ ]:
print(data.head(10))
print(data.tail(10))

Create a plot of the different collums using matplotlib.

Plotting with matplotlib.pyplot is very similar to plotting in matlab. Some key commands:
* <b>plt.figure()</b>: creates a new, blank figure. This changes the current graph and axis to the newly created ones. All commands applied hereafter will be applied to the new, current figure.
* <b>plt.show()</b>: show the figure. If you don't execute this command, no figure will show.
* For more commands, see http://matplotlib.org/users/pyplot_tutorial.html

In [ ]:
start = datetime.datetime(2016, 1, 1, 0, 0)
end = datetime.datetime(2016, 1, 14, 23, 45)

plt.figure()
plt.subplot(311)
plt.plot(data.belpex[start:end], label='belpex')
plt.legend(frameon=False)
plt.subplot(312)
plt.plot(data.solar[start:end], label='solar')
plt.legend(frameon=False)
plt.subplot(313)
plt.plot(data.wind[start:end], label='wind')
plt.legend(frameon=False)
plt.show()

In [ ]:
plt.figure()
data.belpex.plot(grid=True)
plt.show()

# 2. Clean the data by removing outliers

Outliers are mostly difficult, if not impossible, to predict. Therefore, we like to remove them. Moreover, sometimes these outliers are not even correct. 

As an example you can check out the belpex price of the year 2011.

In [ ]:
df = ElexysBelpexFetcher().fetch()
df[df.index.year == 2011].plot()

Read why this happened in this study of the CREG: http://www.creg.be/nl/publicaties/study-on-price-spike-on-belpex-dam-28-march-2011 .

Clue: Use a generic method from statistics that is independent of the timeseries

In [ ]:
mean = data.belpex.mean()
std = data.belpex.std()
n_std = 5
data['belpex'][(data.belpex >= mean + n_std*std)] = mean + n_std*std 
data['belpex'][(data.belpex <= mean - n_std*std)] = mean + n_std*std 

In [ ]:
plt.figure()
data.belpex.plot(grid=True)
plt.show()

# 3. Create boxplot

It is always useful to get more insight in the data yiu are using.

Create a boxplot grouped by
* days in the week
* months
* hours
* years

http://pandas.pydata.org/pandas-docs/version/0.13.1/visualization.html

A pandas dataframe has a table like structure. The very first column in the table contains the indices, uniquely identifying each row in the table. These indices can be a set of integers, or like in this case, a set of timestamps, points in time when the specified data was realised.

'data.index' will return all timestamps of the dataset as an array. These timestamps have certain attributes available. For a reference, see:
http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.html

Below, attributes of the datetime indices are stored in a new column of the table. This allows us to create box plots grouped by these attributes, such as day of the week, hour, etc.

In [ ]:
data['week_days']=data.index.weekday
data['month_days']=data.index.day
data['hours']=data.index.hour
data['months']=data.index.month

data.boxplot(column='belpex', by='month_days')

In [ ]:
data.boxplot(column='belpex', by='week_days')

In [ ]:
data.boxplot(column='belpex', by='hours')

In [ ]:
data.boxplot(column='belpex', by='months')

Finally, <b>give it a try yourself</b>. Make a boxplot grouped by the year.

# 4. Check for randomness in the time series using an autocorrelation plot

This is done by computing autocorrelations for data values at varying time lags. If time series is random, such autocorrelations should be near zero for any and all time-lag separations. If time series is non-random then one or more of the autocorrelations will be significantly non-zero. 

The autocorrelation is the correlation of a timeseries with a delayed copy of itself and is given by:
\begin{equation}
R(\tau) = \frac{E[(X_{t}-\mu)(X_{t+\tau}-\mu)]}{\sigma^2}
\end{equation}
where $X_t$ is the time series and $\tau$ is the time-lag.

How to make an autocorrelation plot using pandas in python:
http://pandas.pydata.org/pandas-docs/stable/visualization.html#visualization-autocorrelation

In [ ]:
# Autocorrelation
lags = np.arange(1, 96*10)
acors = []
for lag in lags:
    acors.append(data.belpex.autocorr(lag))
plt.figure()
plt.plot(lags/4/24.0, acors)
plt.xlabel('Time lag in days')
plt.grid(True)

# 5. Create a scatter matrix 

<b>Why use a scatter plot matrix?</b>

You have three types of data:
* electricity price $p$
* solar generation $s$
* wind generation $w$

For certain moments in time, we have three data points. This means we could think of our dataset as points in a four-dimensional space:
$$\begin{Bmatrix}t\\p\\s\\w\end{Bmatrix}$$

Unless you can think of your dataset in four dimensions, let's make it a bit easier. First, we strip away the time dimension. This leaves us with a more dense three-dimensional space containing all our samples. Still, we don't like to think in three dimensions. So let's just try all possible pairs of the remaining dimensions.

We visualize each pair with a scatter plot. A scatter plot is simply a graph with dots, which are not connected. Hence, we get a matrix of scatter plots, a scatter matrix.

We do all of this to explore our dataset, and find relationships in our data. For example, if the 'price vs solar generation' scatter plot shows dots centered around a straight line, then we can conclude that there is a linear relationship between price and solar generation. This is something that we could exploit later on, when we make our model. 

In [ ]:
import pandas as pd
a = pd.scatter_matrix(data[['belpex', 'solar', 'wind']])

If the diagonal is not what you would expect, then you are right. For more information, see

http://stackoverflow.com/questions/26360759/understanding-the-diagonal-in-pandas-scatter-matrix-plot

# 6. Linear regression

Metric to define the quality of the foreceast

In [ ]:
def get_accuracy(x, y):
    return np.mean(np.abs(x - y))/np.mean(x)

In this section, we will use the scikit learn toolbox. For more information, see
* [API reference](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection)
* [Tutorials](http://scikit-learn.org/stable/tutorial/index.html)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
model = LinearRegression(normalize=True)
lags = [24*4] + list(range(96*7, 96*35, 96))
features = ['wind', 'solar']
print(lags)
len(lags)

In [ ]:
index = data.index 
for lag in lags:
    data['belpex_lag_{}'.format(lag)] = data.belpex.shift(lag)
    features.append('belpex_lag_{}'.format(lag))
data['day_in_year'] = data.index.dayofyear
data = data.ix[index, ]
data = data.dropna()

X = data[features]
y = data.belpex

cv = KFold(n=len(data), n_folds=10, shuffle=False)
data['belpex_pred'] = cross_val_predict(model, X, y, cv=cv, n_jobs=-1)

print(get_accuracy(data.belpex, data.belpex_pred))


In [ ]:
print(features)

In [ ]:
start = datetime.datetime(2016, 1, 1, 0, 0)
end = datetime.datetime(2016, 1, 7, 23, 45)

plt.figure()
data.belpex[start:end].plot(grid=True)
data.belpex_pred[start:end].plot(grid=True)
plt.legend(frameon=False,loc=2)
plt.show()

# 7. Neural network (a naive implementation)

In [ ]:
import os

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import RMSprop, SGD

Create input and output training data. <br>
We will just predict the next day, based on the previous day.

In [ ]:
n_hours = 24
start = datetime.datetime(2016, 1, 1, 0, 0)
end = datetime.datetime(2016, 12, 29, 23, 45)
X = data['belpex'][start:end].resample('1H').mean().reshape(-1, n_hours)


start = datetime.datetime(2016, 1, 2, 0, 0)
end = datetime.datetime(2016, 12, 30, 23, 45)
Y = data['belpex'][start:end].resample('1H').mean().reshape(-1, n_hours)

In [ ]:
print(data['belpex'][start:end].resample('1H').mean().shape)

In [ ]:
print('input_features ' + str(X.shape))
print('target diminsions ' + str(Y.shape))

Create neural network with one hidden layer

In [ ]:
neurons = [24, 24 ]
activation_functions = ['relu', 'linear']

model = Sequential()
model.add(Dense(neurons[0], input_dim=X.shape[1], activation=activation_functions[0]))
model.add(Dense(neurons[1], activation=activation_functions[1]))
    
rprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-6)
model.compile(loss='mean_squared_error', optimizer=rprop)

In [ ]:
output_training = model.fit(X, Y, epochs=500, batch_size=32, verbose=0)
mse = output_training.history['loss'][-1]
print('- mse is %.4f' % mse + ' @ ' + str(len(output_training.history['loss'])))

In [ ]:
predict_nn = model.predict(X)

print(get_accuracy(Y.flatten(), predict_nn.flatten()))


In [ ]:
# Plots
plt.figure()
plt.plot(Y[:10,:].flatten(), color='blue', label='actual price')
plt.plot(predict_nn[:10,:].flatten(), color='red', label='forecast NN')
plt.legend(frameon=False)
plt.show()

# 8. Assignment: Improve the naive implementation in Step 7


* Create test and validation set
    * https://en.wikipedia.org/wiki/Test_set
    * http://stats.stackexchange.com/questions/19048/what-is-the-difference-between-test-set-and-validation-set
* Add additional features
    * same day of the previous week (see autocorrelation plot)
    * use a wind and solar forecast as an additional feature (assume you can make perfect forecasts)
    * Use EliaTotalLoad from datafetcher
    * Are there other features that have an influence on the price and thus may affect prediction performance
* Tune the structure of the neural network (use grid search)
    * number of layers
        * http://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
    * different activation functions
        * https://en.wikipedia.org/wiki/Activation_function

Advice: read the documentation on https://keras.io/ for help